In [93]:
# Import libraries 
import cv2
from mediapipe.python.solutions import face_detection, face_mesh, drawing_utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import mediapipe.framework.formats.landmark_pb2 as mp_landmark

In [100]:
image_path = '/Users/lancelotbosseler/code/labosseler/final-project/final-attention/notebooks/trial-images/no-face.jpeg'

In [101]:
#Face detection 
def face_detector(image_path: str) -> list:                           # output = list of ndarrays  
    with face_detection.FaceDetection(
        model_selection=1, min_detection_confidence=0.5) as face_det:
        faces = []
        les_scores = []
        image_array = cv2.imread(image_path)  
        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
        image_rgb = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
        results = face_det.process(image_rgb)
        if results.detections: 
            for i, detection in enumerate(results.detections): 
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = image_rgb.shape
                x1 = int(bbox.xmin * w) - int(0.05 * int(bbox.xmin * w))
                y1 = int(bbox.ymin * h) - int(0.05 * int(bbox.ymin * h))
                x2 = int((bbox.xmin + bbox.width) * w) + int(0.05 * int(bbox.xmin * w))
                y2 = int((bbox.ymin + bbox.height) * h) + int(0.05 * int(bbox.ymin * h))
                cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                face = image_rgb[y1:y2, x1:x2]
                face = cv2.resize(face, (400, 400))
                faces.append(face)
                les_scores.append(detection.score)
    return faces  #, les_scores                     
        

In [102]:
# Meshes detection 
def mesh_detection(face: np.ndarray) -> mp_landmark.NormalizedLandmarkList:
    with face_mesh.FaceMesh(
            static_image_mode=True,
            max_num_faces=10,
            min_detection_confidence=0.5,
            refine_landmarks=True,
            min_tracking_confidence=0.5) as face_meshe:
        results = face_meshe.process(face)
        if results.multi_face_landmarks:
                face_landmarks = results.multi_face_landmarks[0]
                #annotated_image = face.copy()
                #drawing_utils.draw_landmarks(annotated_image, face_landmarks)
                #plt.imshow(annotated_image)
                return face_landmarks
        
        return None

In [103]:
#Everything in a single function 
def image_process(image: Image) -> list:       #list of NormalizedLandmarkList
    
    #Taking the output list of face_detector 
    faces = face_detector(image)
    
    #Loop trough the list to take one face by one face and add it into a list 
    meshed_coord_faceslist = []
    for face in faces:
        mesh_coordinates = mesh_detection(face)
        if mesh_coordinates:
            meshed_coord_faceslist.append(mesh_coordinates)
    
    # If no face detected return None
    if not meshed_coord_faceslist:
        return None
    
    return meshed_coord_faceslist
        

In [104]:
landmarks_faces = image_process(image_path)
# Now you can do a loop on landmarks to apply iris function on one face a the time
# for face in landmarks_faces:
        # your function 

In [105]:
landmarks_faces